In [1]:
import numpy as np
import pandas as pd

import pickle

In [2]:
A_matrices = dict()
B_matrices = dict()
pi_matrices = dict()

In [3]:
data = pd.read_csv('malware_data.csv', index_col = 0)

In [4]:
data.head()

,OpCode Sequence,Malware Family
0,push mov push mov mov mov mov cmp jz push sti ...,CLUSTERclarkclark
1,push add mov push call test jz movzx mov add p...,ufasoftbitcoin
2,push push push call add push call mov push pus...,ufasoftbitcoin
3,push add mov push call test jz movzx mov add p...,ufasoftbitcoin
4,mov push mov push push mov and test push push ...,CLUSTERgdata


In [5]:
counts_family = data.groupby('Malware Family')['Malware Family'].transform(len)
mask = (counts_family > 50)
data = data[mask]
malware_families = data["Malware Family"].unique()

In [6]:
for i in range(len(malware_families)):
    with open('HMMMatrices/' + malware_families[i] + 'A.pkl', 'rb') as f:
        A_matrices[malware_families[i]] = pickle.load(f)
    with open('HMMMatrices/' + malware_families[i] + 'B.pkl', 'rb') as f:
        B_matrices[malware_families[i]] = pickle.load(f)
    with open('HMMMatrices/' + malware_families[i] + 'Pi.pkl', 'rb') as f:
        pi_matrices[malware_families[i]] = pickle.load(f)
    

In [7]:
data.head()

,OpCode Sequence,Malware Family
18,push mov sub mov mov and mov and mov inc mov c...,zeroaccess
19,push mov sub push push push mov mov mov mov mo...,zeroaccess
20,push mov sub jmp push mov jmp inc cli mov mov ...,zeroaccess
21,push mov push dec push push mov push call push...,zeroaccess
22,push mov sub push and jmp mov inc mov cmp jnb ...,zeroaccess


In [8]:
for index, row in data.iterrows():
    if len(row['OpCode Sequence'].split(' ')) < 100:
        data = data.drop(index)

In [9]:
import pickle
with open('word2vec_embeddings.pkl', 'rb') as fp:
    embed = pickle.load(fp)

word_to_idx = dict()
index = 0
for word in embed:
    word_to_idx[word] = index
    index = index + 1

In [10]:
class OptimalHiddenStateGetter:
    def __init__(self, obs_seq: str, word_to_idx: dict(), malware_family: str, vocab_size: int, num_hidden_states: int, A_matrices: dict(),
                B_matrices: dict(), pi_matrices: dict()):
        self.obs = []
        for word in obs_seq:
            self.obs.append(word_to_idx[word])
        self.T = len(self.obs)
        self.N = num_hidden_states
        self.M = vocab_size
        self.alpha = np.zeros((self.T, self.N))
        self.beta = np.zeros((self.T, self.N))
        self.gammas = np.zeros((self.T, self.N))
        self.digammas = np.zeros((self.T, self.N, self.N))
        self.A = A_matrices[malware_family]
        self.B = B_matrices[malware_family]
        self.pi = pi_matrices[malware_family]
        self.c = np.zeros((self.T))
        
    def forward_algorithm(self):
        for i in range(self.N):
            self.alpha[0][i] = self.pi[i]*self.B[i][self.obs[0]]
            self.c[0] = self.c[0] + self.alpha[0][i]
        
        self.c[0] = (1/self.c[0]*1.0)
        
        for i in range(self.N):
            self.alpha[0][i] = self.c[0]*self.alpha[0][i]
        
        for t in range(1, self.T):
            for i in range(self.N):
                for j in range(self.N):
                    self.alpha[t][i] += self.alpha[t-1][j]*self.A[j][i]
                self.alpha[t][i] = self.alpha[t][i]*self.B[i][self.obs[t]]
                self.c[t] += self.alpha[t][i]
            
            self.c[t] = (1/self.c[t]*1.0)
            
            for i in range(self.N):
                self.alpha[t][i] = self.c[t]*self.alpha[t][i]
        
        return self.c
    
    def backward_algorithm(self):
        for i in range(self.N):
            self.beta[self.T - 1][i] = self.c[self.T-1]
        
        for t in reversed(range(self.T - 1)):
            for i in range(self.N):
                for j in range(self.N):
                    self.beta[t][i] = self.beta[t][i] + self.A[i][j]*self.B[j][self.obs[t+1]]*self.beta[t+1][j]
                self.beta[t][i] = self.c[t]*self.beta[t][i]
                
    def find_optimal_hidden_state(self):
        print("running forward algorithm")
        self.forward_algorithm()
        print("running backward algorithm")
        self.backward_algorithm()
        print("finding optimal hidden state")
        hidden_state_seq = np.zeros((self.T))
        for t in range(self.T):
            max_prob = 0
            for i in range(self.N):
                if self.alpha[t][i]*self.beta[t][i] > max_prob:
                    max_prob = self.alpha[t][i]*self.beta[t][i]
                    hidden_state_seq[t] = i
        return hidden_state_seq
            

In [12]:
num_hidden_states = 20
cell_state_dict = dict()
for index, row in data.iterrows():
    print('Current index running is: {}'.format(index))
    obs_seq = row['OpCode Sequence'].split(' ')[0:100]
    hmm = OptimalHiddenStateGetter(obs_seq, word_to_idx, row['Malware Family'], len(word_to_idx), num_hidden_states,
                                  A_matrices, B_matrices, pi_matrices)
    cell_state_dict[index] = hmm.find_optimal_hidden_state()

Current index running is: 18
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 19
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 20
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 21
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 22
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 23
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 24
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 25
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 26
running forward algorithm
running backward algorithm
finding optimal hidden state
C

finding optimal hidden state
Current index running is: 93
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 94
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 95
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 96
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 97
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 98
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 99
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 100
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 101
running forward algorithm
running backward algorithm

running backward algorithm
finding optimal hidden state
Current index running is: 167
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 168
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 169
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 170
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 171
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 172
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 173
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 174
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 175
running forward al

running backward algorithm
finding optimal hidden state
Current index running is: 242
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 243
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 244
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 245
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 246
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 247
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 248
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 249
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 250
running forward al

running backward algorithm
finding optimal hidden state
Current index running is: 316
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 317
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 318
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 319
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 320
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 321
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 322
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 323
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 324
running forward al

...

finding optimal hidden state
Current index running is: 7830
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7831
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7832
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7833
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7834
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7835
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7836
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7837
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7838
running forward algorithm
running ba

finding optimal hidden state
Current index running is: 7903
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7904
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7905
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7906
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7907
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7908
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7909
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7910
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7911
running forward algorithm
running ba

running backward algorithm
finding optimal hidden state
Current index running is: 7995
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7996
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7997
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7998
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 7999
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8000
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8001
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8002
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8003
running f

finding optimal hidden state
Current index running is: 8127
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8128
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8129
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8130
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8131
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8132
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8133
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8134
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8135
running forward algorithm
running ba

running backward algorithm
finding optimal hidden state
Current index running is: 8238
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8239
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8240
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8241
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8242
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8243
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8244
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8245
running forward algorithm
running backward algorithm
finding optimal hidden state
Current index running is: 8246
running f

In [17]:
with open('cell_state_dict.pkl', 'wb') as fp:
    pickle.dump(cell_state_dict, fp)

In [18]:
cell_state_dict_norm = dict()
for index in cell_state_dict:
    cell_state_dict_norm[index] = np.linalg.norm(cell_state_dict[index])

In [19]:
with open('cell_state_dict_norm.pkl', 'wb') as fp:
    pickle.dump(cell_state_dict_norm, fp)